In [ ]:
import chromdyn
from chromdyn import (
    ChromatinDynamics,
    TopologyGenerator,
    HiCManager, save_pdb,
    PlatformManager,
    Analyzer,
    TrajectoryLoader,
)
hicman = HiCManager()

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

In [ ]:
#Initialize generator object
generator = TopologyGenerator()

#generate topology
generator.gen_top([100])

In [ ]:
#Initialize simulation object
#output will be generated if not present
sim = ChromatinDynamics(generator.topology, name = 'test', platform_name = "OpenCL", output_dir = "output", console_stream = True)

In [ ]:
#Add forces
sim.force_field_manager.add_harmonic_bonds()
sim.force_field_manager.add_self_avoidance()
sim.force_field_manager.add_flat_bottom_harmonic()

In [ ]:
#set up the simulation
sim.simulation_setup(
            init_struct='randomwalk',
            integrator='active-langevin',
            temperature=120.0,
            timestep=0.01,
            save_pos=True,
            save_energy=True,
            energy_report_interval=3_000,
            pos_report_interval=1000,              
            )

In [ ]:
F_seq=np.zeros(100)
F_seq[50:60]=2.0
tau_seq=[1000.0]*sim.num_particles

sim.set_activity(F_seq=F_seq, tau_seq=tau_seq)

In [ ]:
#collapse run
sim.run(10_000, report=True)

In [ ]:
sim.print_force_info()

In [ ]:
p=sim.get_active_params()

p.keys()

In [ ]:
save_pdb(sim)

In [ ]:
sim.save_reports()

In [ ]:
#Generating HiC from traj
hic = hicman.gen_hic_from_cndb(traj_file=sim.reporters.get('position').filename, mu=2.0, rc=2.0, p=4.0, parallel=True)
# np.savetxt(os.path.join(str(sim.output_dir), f"Pi_{str(replica)}.txt"), hic)

In [ ]:
plt.matshow(np.log10(hic), cmap='Reds')
plt.colorbar()

In [ ]:
from chromdyn.traj_utils import TrajectoryLoader

In [ ]:
data=TrajectoryLoader.load(sim.reporters.get('position').filename)

data.shape

In [ ]:
# Set up the 3D figure and axis
fig = plt.figure(figsize=(5,5), dpi=300)
ax = fig.add_subplot(111, projection='3d')

# Set axis limits (adjust as needed)
d=10.0
ax.set_xlim(-d,d)
ax.set_ylim(-d,d)
ax.set_zlim(0,2*d)

# Initial scatter plot
sc = ax.scatter([], [], [], s=50)

# Update function for each frame
def update(frame):
    # for coll in ax.collections:
        # coll.remove()
    ax.clear()
    positions = data[frame]
    
    ln = ax.plot(positions[:, 0], positions[:, 1], positions[:, 2], lw=2, c='C4')
    
    ax.set_xlim(-d,d)
    ax.set_ylim(-d,d)
    ax.set_zlim(0,2*d)
    return ln,

# Create the animation
anim = FuncAnimation(fig, update, frames=data.shape[0], interval=150, blit=False)

# HTML(anim.to_html5_video())